In [1]:
import mediapipe as mp
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
import os
import cv2
import math
import numpy as np
import copy
import itertools
import csv
from collections import Counter
from collections import deque
from utils import CvFpsCalc
from matplotlib import pyplot as plt

In [2]:
DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480


def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  return img

def load_images_from_folder(folder):
    images = []
    
    for filename in os.listdir(folder):
        #print(filename)
        os.chdir(folder)
        img = cv2.imread(filename)
        #print(img)
        if img is not None:
            img=resize_and_show(img)
            images.append(img)
    return images

In [3]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

In [4]:
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))
    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

In [57]:
def logging_csv(number, mode, landmark_list, point_history_list):
# if mode == 0:
#     pass
# if mode == 1 and (0 <= number <= 9):
    csv_path = 'model/keypoint_classifier/keypoint_one_frame_model_left.csv'
    #csv_path = 'model/keypoint_classifier/keypoint_withoutQ&X.csv'


    with open(csv_path, 'a', newline="") as f:
        writer = csv.writer(f)
        writer.writerow([number, *landmark_list])

    return

In [14]:
directory = 'C:\\Users\\User\\Desktop\\one_frame_model\\moving_frames_left'
target = 'C:\\Users\\User\\Desktop\\one_frame_model\\one_frame_model_left'

In [15]:
for letter in os.listdir(directory):
    let = os.path.join(directory, letter)
    t1 = os.path.join(target, letter)
    os.mkdir(t1)
    for number in os.listdir(let):
        num = os.path.join(let, number)
        for filename in os.listdir(num):
            # checking if it is a file
            new_file = number + "_" + filename
            #print(new_file)
            tmp = os.path.join(num, filename)
            if os.path.isfile(tmp):
                image= cv2.imread(tmp)
                cv2.imwrite(os.path.join(t1, new_file), image)
            break

In [25]:
# augmentation
from keras.preprocessing.image import ImageDataGenerator
from skimage import io

# Construct an instance of the ImageDataGenerator class
# Pass the augmentation parameters through the constructor. 

datagen = ImageDataGenerator( 
        width_shift_range=0,   #% shift
        height_shift_range=0,
        zoom_range=0.2,
        horizontal_flip=False,
        fill_mode='constant', cval=125)    #Also try nearest, constant, reflect, wrap

In [43]:
directory_aug = 'C:\\Users\\User\\Desktop\\one_frame_model\\one_frame_model'
tmp = 'C:\\Users\\User\\Desktop\\one_frame_model\\copies\\not_used_letters_left'

for letter in os.listdir(directory_aug):
    let = os.path.join(directory_aug, letter)
    t1 = os.path.join(tmp, letter)
    os.mkdir(t1)

In [52]:
import shutil
dir = 'C:\\Users\\User\\Desktop\\one_frame_model\\copies\\not_used_letters_left'

for letter in os.listdir(dir):
    let = os.path.join(dir, letter)
    one = os.path.join(let, '1')
    os.mkdir(one)
    for filename in os.listdir(let):
        if(os.path.isfile(os.path.join(let, filename))):   
            shutil.move(os.path.join(let, filename),one)

In [54]:
directory_aug = 'C:\\Users\\User\\Desktop\\one_frame_model\\copies\\not_used_letters_left'
target_aug = 'C:\\Users\\User\\Desktop\\one_frame_aug\\one_frame_aug_left'


for letter in os.listdir(directory_aug):
    let = os.path.join(directory_aug, letter)
    t1 = os.path.join(target_aug, letter)
    print(let)
    i = 0
    for batch in datagen.flow_from_directory(directory=let,
                                         target_size=(480,480),
                                         color_mode="rgb",
                                         save_to_dir=t1, 
                                         save_prefix='aug_woman', 
                                         save_format='png'):
        i += 1
        if i > 4:
            break


C:\Users\User\Desktop\one_frame_model\copies\not_used_letters_left\A
Found 1 images belonging to 1 classes.
C:\Users\User\Desktop\one_frame_model\copies\not_used_letters_left\B
Found 1 images belonging to 1 classes.
C:\Users\User\Desktop\one_frame_model\copies\not_used_letters_left\C
Found 1 images belonging to 1 classes.
C:\Users\User\Desktop\one_frame_model\copies\not_used_letters_left\E
Found 1 images belonging to 1 classes.
C:\Users\User\Desktop\one_frame_model\copies\not_used_letters_left\I
Found 1 images belonging to 1 classes.
C:\Users\User\Desktop\one_frame_model\copies\not_used_letters_left\L
Found 1 images belonging to 1 classes.
C:\Users\User\Desktop\one_frame_model\copies\not_used_letters_left\M
Found 1 images belonging to 1 classes.
C:\Users\User\Desktop\one_frame_model\copies\not_used_letters_left\N
Found 1 images belonging to 1 classes.
C:\Users\User\Desktop\one_frame_model\copies\not_used_letters_left\O
Found 1 images belonging to 1 classes.
C:\Users\User\Desktop\one_fr

In [58]:
#Land mark collection
root_dir="C:\\Users\\User\\Desktop\\one_frame_aug\\one_frame_aug_left"
letters=os.listdir(root_dir)
letters.sort()
label=-1
for letter in letters:
    label=label+1
    input_dir =os.path.join(root_dir,letter)
    images=load_images_from_folder(input_dir)
    os.chdir("C:\\Users\\User\\Desktop\\Sign_language\\PolishAlphabet_SignLanguage_Detection")
    first = True
    for image in images:
        with mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=2,
            min_detection_confidence=0.7) as hands:
        
            results = hands.process(cv2.flip(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 1))

            tmp = "No"
            index = 0

            ###############################
            # RIGHT HAND DETECTION
            if (results.multi_hand_landmarks is not None):
                for idx,hand in enumerate(results.multi_handedness):
                    #print(hand.classification[0].label)
                    if(str(hand.classification[0].label)=="Left"):
                        # print(hand.classification[0].label)
                        # print(idx)
                        index = idx
                        break
            ###############################
            # print(index)

            if (results.multi_hand_landmarks is not None):
                # Landmark calculation
                landmark_list = calc_landmark_list(image, results.multi_hand_landmarks[index])


                # Conversion to relative coordinates / normalized coordinates
                pre_processed_landmark_list = pre_process_landmark(
                        landmark_list)

                # print(pre_processed_landmark_list)
                # Write to the dataset file
                logging_csv(label, 1, pre_processed_landmark_list,[]) 